In [61]:
import re
import ast
import json
import pandas as pd
from pprint import pprint

from url_parser import parse_url, get_url, get_base_url

In [89]:
def get_size():
    df = pd.read_csv('1.csv')
    print(df.shape)
    df = pd.read_csv('1_label.csv')
    print(df.shape)
    df = pd.read_csv('1_nolabel.csv')
    print(df.shape)
    

def get_iab_categories(iab_categories):
    iab_categories_list = []
    iab_categories = ast.literal_eval(iab_categories)
    for iab_category in iab_categories:
        for value in list(iab_category.values())[:-1]:
            iab_categories_list.append([value, iab_category['score']])
    return iab_categories_list

    
def get_path(url):
    path = re.sub('[^A-Za-z]+', ' ', parse_url(url)['path']).strip()
    return path
    
def clean():
    # for i in tqdm(range(1, 10)):
    df = pd.read_csv('test.csv')
    
#     df_nolabel = df[df['iabCategories'].isnull()]
#     df_nolabel.to_csv('test_nolabel.csv')
    
    df = df[~df['iabCategories'].isnull()]
    df['iabCategories'] = df['iabCategories'].apply(get_iab_categories)
    df = df.explode('iabCategories', ignore_index = True)
    split = pd.DataFrame(df['iabCategories'].to_list(), columns = ['iab_categories', 'confidence'])
    df = pd.concat([df, split], axis = 1)
    df = df.drop('iabCategories', axis = 1)
    df['base_url'] = df['url'].apply(get_base_url)
    df['domain'] = df['url'].apply(lambda url: parse_url(url)['domain']) 
    df['path'] = df['url'].apply(get_path)
    
    df.to_csv('test_label.csv')
    return df
    

In [90]:
df = clean()
# get_size()

In [91]:
df.head(10)

,Unnamed: 0,browserFamily,channel,deviceType,os,refDomain,url,iab_categories,confidence,base_url,domain,path
0,0,Mobile Safari,linkedin,Smartphone,iOS,www.linkedin.com,https://www.facs.org/for-medical-professionals...,health_&_fitness,high,https://www.facs.org,facs,for medical professionals news publications ne...
1,1,Mobile Safari,facebook,Smartphone,iOS,m.facebook.com,https://www.rhinews.com/actualites/proposition...,education,high,https://www.rhinews.com,rhinews,actualites proposition de reparation creative ...
2,1,Mobile Safari,facebook,Smartphone,iOS,m.facebook.com,https://www.rhinews.com/actualites/proposition...,"law,_gov't_&_politics",high,https://www.rhinews.com,rhinews,actualites proposition de reparation creative ...
3,1,Mobile Safari,facebook,Smartphone,iOS,m.facebook.com,https://www.rhinews.com/actualites/proposition...,education,high,https://www.rhinews.com,rhinews,actualites proposition de reparation creative ...
4,1,Mobile Safari,facebook,Smartphone,iOS,m.facebook.com,https://www.rhinews.com/actualites/proposition...,special_education,high,https://www.rhinews.com,rhinews,actualites proposition de reparation creative ...
5,1,Mobile Safari,facebook,Smartphone,iOS,m.facebook.com,https://www.rhinews.com/actualites/proposition...,careers_education,high,https://www.rhinews.com,rhinews,actualites proposition de reparation creative ...
6,2,Chrome,NaN,Personal computer,Windows,NaN,https://pelispgo.com/,society,high,https://pelispgo.com,pelispgo,
7,2,Chrome,NaN,Personal computer,Windows,NaN,https://pelispgo.com/,family_&_parenting,high,https://pelispgo.com,pelispgo,
8,2,Chrome,NaN,Personal computer,Windows,NaN,https://pelispgo.com/,hobbies_&_interests_video_&_computer_games,high,https://pelispgo.com,pelispgo,
9,6,Chrome Mobile,NaN,Smartphone,Android,NaN,http://incestflix.win/tag/SloansMoans/page/3,society,high,http://incestflix.win,incestflix,tag SloansMoans page


In [20]:
from urllib.parse import urlparse
from url_parser import parse_url, get_url, get_base_url

In [85]:
urls = [
    'https://www.facs.org/for-medical-professionals/news-publications/news-and-articles/acs-brief/january-16-2024-issue/surgeon-well-being-coalition-expands-resources-throughout-specialty/?utm_campaign=newsletter-bulletin-brief&utm_medium=social&utm_source=facebook', 
    'https://shop.consumerbags.com/?cep=szJ4itggB-mMFq_chCl8AELcIb_2hXZrPLZioIRxzdeKXYXd6ECO3DC9xUE7UfUyRM0t1wtKcMt_PpLKHhAharO51xmexjlUw1GMUGTI3M5qLHfysbipTmdQr4Mv6aZauTKAXI3cURKeuXP2m7wQnVVXVK7lrSIZy2DW3bsguQp3A2wHtHi68rdWKf5bijPD0IaRdPekL1A_G3lWiYht-Ezl-YfhmWo75euMkY66zUz0NVQkWKIocfSwkBWcdkTrM9lq9MDYu18dhWvtScpUZ0PsAwziUOlJq-nfIiJI-pN8fxpnBIKSIHG_yRiop2ikAXWxDViEEjAmxduLeN2-1CtuMMHR0wEdXqxrIz9Cob90724jeYljnakNno9vbP2MF_B0olGC43ApMv4HxEZoMCM4z4CVbAYLRpcVtpvR6z0Tvj7CJ7JpOtukPsO_p4cR_Xmr6FXZRxrGz6KP0SCGww8JQyLokDrQGBXXAKJZE5aR5NH17mbDaRpL9tiJ8O74VuR0oqtEOjXc-Ea4kZRAIEcLygQ8nRLLR5I17Au9X8_hvhUeHFX53dJOQQUpIDyoYP1yFx_O7j0ipnwSKlF8jhb24Jv_MgQN45tOqhQMivS3fscXAZutqml33oRoN429IdKs7PVn0QEslu_ffPHy4Bp24WDEKRUUwxV1QwmxcAKqE_NM0Ssv4ohbdRUucIgHNVWxaSMYR9mb9wLe5NyAmLVlSCZzBjuXGByk95DXx1FdYhgpBOKpgvPEQqMTXed29rtbsxMlC3IGomyhgBiMzgIZovw7TpuBSLyU4rUMyFGq7Kmcc6lxpGL7YM0TTrnQ4QtIV1HxXGmw3tX7YP3yHl-MffmyUNEz8zri_ULqzSyk_265g4IO2ku-gQ9ALpAZE7C5wKSvJPlE_Sdo-sV2FJNXEGqva9zUUAYTd6BtcYwn4EbcY22ZxktTni1TP-au33jaD5h57WDnFAgZrBJjUjNUZj7ADQknSu0S2Eyxne1ae8_-4qR2XKdQ3WvrwADN6YICE12AOLmtG-dOLLL8RX8ljRQS5030fkIWyJLsGUWvTUsXq_HDCYcz6XFg57dW5zMtJh2g69iYnNv5yY53hamQfP3u_Yh5BJyvL_9HFutnMt5qKi9WIr3H5EcdQBAlziqJ5cui3IBZgtMw_O04zqt7jurubM4hzTPNxTxtowkWShzZy6lcj-EoAgZGWWFYg0K-xtzqyCQ0SZOLE5SoxJ3NQJiOKNyVmy8LfzBo1SUMvBdgG_cX2krr-GTd1CNPA4_tjxKn36j25yTXtRJnCkcS9tb20Fu9NRqVcDTXRb0X9NVVgXH5a3C9JEINseHRcPCIoWPs7WeMW-dpa8vRwlVPjPkNbPODkMtc3-XARgxHXrwE4TSiirrwnurlZ-PfPVtTuy27d3C8QJ-LxgLeLHwDeBxmkpgxESixMgupCExUlXZH2Olb8N3yXEpGUoe_Wmzti-VywCDxVmQGMiLEC62oghbCk_ST96UT2OomIdWDTjDTRcqX0UdjHGfFCB__oyEIfGLgUN4wyM-t52HV2C1tYWOl-5wLHqyGKEtw9KqNEqndP2yawo_gdM-ictYzazq2FSXeCqMPe_lMTAGMeMQWGPp6agoCtc1Tp57mak6bbZvoHPdYHGVQD52R6OAewXqC7S9t1VrII1G2BKVcpLxlS9eGWoTPK5gnQgWlkEs29LkHLUkOn9weq6rso2q1cs3h9y0dxQSJOyIoI1msfTf3aAxXdLo_igNRCnojtv_GkSdqqGokE7kT9N1qqAVL7FtxwxqcyUVMzPZUkZONtwQGhQQbt5yGQr0XG95Wtt52SJdKAHJg00CT5w6N5DG4RF-a9RHaRtsJ_4fep-DvLxqI8uEuIqRPj-WO6_s91z70We_6NX8WN_FtjsbRd1COREu5dbxEfRCEhdtWcmcOnfR1XAscAvKuGq2tSc31jrF3iL7Qh6ZddXSaMKTFiz0HRhJ6EThv0G11nwU8Azu0Ww&lptoken=175905207611317641a8&liveclick=f0d90105-1d5c-3da9-a8c9-e8a93a63c0bf&li_did=f0d90105-1d5c-3da9-a8c9-e8a93a63c0bf',
    'https://thecelebinsider.com/patricia-richardson-who-played-mom-jill-taylor-on-home-improvement-showed-off-her-ageless-look-at72-with-long-gray-hair-at-her-twin-daughters-wedding/?fbclid=IwAR1RnlEDN69lM6JVuf6-sPLsW1iNWRi5wJuc6nopY07EQWAAC7x0zkxUu74_aem_AT8Z0cx1r4dU7c2rpvn-8lSQa4b5Q2wsB1tY5OOuthLKt1yBj8AgjnrMdlSuSZoGzYs'

]

In [86]:
for url in urls:
    get_path(url)

/for-medical-professionals/news-publications/news-and-articles/acs-brief/january-16-2024-issue/surgeon-well-being-coalition-expands-resources-throughout-specialty/
for medical professionals news publications news and articles acs brief january issue surgeon well being coalition expands resources throughout specialty
/

/patricia-richardson-who-played-mom-jill-taylor-on-home-improvement-showed-off-her-ageless-look-at72-with-long-gray-hair-at-her-twin-daughters-wedding/
patricia richardson who played mom jill taylor on home improvement showed off her ageless look at with long gray hair at her twin daughters wedding
